<a href="https://colab.research.google.com/github/Noah-Himselff/Persian_Sentiment_Analysis/blob/main/notebookcfaa1b5f27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'stop-words:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4474994%2F7672068%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T133053Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9b5812ff13a1dc28618aca439de61cc5405afcdbd4dd17ee98cfc56d1d0ca4f392a4d4d0e207e2d8d769e5b88bc967effe8fd8413cbabbf377e4bc83db719fc1ccb2f7a535bf49a1c85e7d01df72bb2fd53ef54fb21fb9248b032dbd891d1462fb1909da8861956f90e38ecdc07233be07df1e51add144a1bc505bfc8d0b6c20dc0eeec43ab3f3f392af035f2ce54195d92888e46693d371b7574dd9d116e5e19582d546d0bc6b7400b21b4c54a67c3d63790e1984b7585b4f9566b56b49b3060c5102e022990c9075aa0321a5c24a004e4b06022041b2a876717da11ad1550e7d3833695a5c4550dfbfb12c3a62f5dbe9e4b7b3e5ba498a47d483f8eb7062b9,test23:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5478761%2F9081045%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T133053Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5dbf30d2d708a57ecc8f9c31e068f3037a5677b2038a279f6d825e09f598adccdb717c8955ccb50e38b8566ea5a914b9680488147ec780df68b03483ca1d157c48f03eeffe41bb3652725fac8bae4cd2bcc88ac8739716a0a4679533f9ef73c6b236abb2a00ff2e1279dbc71ede799b45fc1e3b14189aac2fd2f87633438e9807a8ce1a0e66a76b0670056e0ee80048c837b78fcff2b3a5d06152337452be6d8e2fae8de88675fa57e12225a18bc3dd0c02f3557bff2a6b2ba5a95b5bb5201c1cf233e56dc68ededdfa7a0d1427a098a791a965444d52c704ce500b0a6c4824e8cab4513eca0eecffbd5638021461d772d501c24dbf16351564d5cf226ba91b7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 1577 bytes downloaded
Downloaded and uncompressed: stop-words
[==================================================] 295440 bytes downloaded
Downloaded and uncompressed: test23
Data source import complete.


In [2]:
%pip install --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [23]:
import json
import pandas as pd
# Path to your JSON file
file_path = '/kaggle/input/test23/lol.json'
# Load the JSON data
with open(file_path, 'r') as f:
    data = json.load(f)
# Assuming data is a list of dictionaries
# Create a DataFrame with 'body' and 'rate' elements
df = pd.json_normalize(data)
# Select only the 'body' and 'rate' columns
df = df[['body', 'rate']]
# Display the DataFrame
print(df.head())

                                                body  rate
0  جنس خوبی داره من برا تهویه اتاق خواب گرفتم راض...     5
1  خوبه و کار راه اندازه ولی برای پایه هاش چه خوب...     3
2                        کاش بزرگتر بود یا چرخش داشت     5
3     دوساله دارمش وخداروشکر مشکلی نداشته وازش راضیم     5
4                                            بد نیست     3


In [33]:
df

,body,rate
0,جنس خوبی داره من برا تهویه اتاق خواب گرفتم راض...,5
1,خوبه و کار راه اندازه ولی برای پایه هاش چه خوب...,3
2,کاش بزرگتر بود یا چرخش داشت,5
3,دوساله دارمش وخداروشکر مشکلی نداشته وازش راضیم,5
4,بد نیست,3
...,...,...
1986,دوستش داشتم خیلی خوب بود,5
1987,ممنون از دیجی کالا,5
1988,کیفیت خوب,4
1989,خوبه قیمت و کیفیت مناسبی داره جمع و جوره واسه ...,4


In [34]:
df['rate'].value_counts()

,count
rate,
5,1091
4,624
3,154
1,79
2,38
0,5


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import ( AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorWithPadding )
import re

In [6]:
def remove_newline(df):
    df['comment'] = df['comment'].str.replace('\n', ' ')
    return df

def remove_consecutive_duplicates(comment_list):
    modified_comments = []

    for text in comment_list:
        result = ""
        previous_char = None

        for char in text:
            if char != previous_char:
                result += char
            previous_char = char

        modified_comments.append(result)

    return modified_comments

def modify_sentences(sentences):
    modified_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if words[-1].endswith('ه'):
            words[-1] = words[-1][:-1]+ ' ' + 'است'
        modified_sentences.append(' '.join(words))
    return modified_sentences

def fix_half_space(text):
    half_space = "\u200C"
    normal_space = " "

    fixed_text = text.replace(half_space, normal_space)
    return fixed_text

def remove_stop_words(comment_column):
    stop_words = final_stop_words
    words = comment_column.str.split()
    words = words.apply(lambda word_list: [word for word in word_list if word not in stop_words.iloc[:, 0].tolist()])
    return words.apply(lambda word_list: ' '.join(word_list))

In [7]:
stop_words=pd.read_csv('/kaggle/input/stop-words/stopwords.dat')

In [8]:
indicies = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,23,31,33,46,49,68,72,77,90,102,109]
final_stop_words = stop_words.loc[indicies]
final_stop_words=final_stop_words.reset_index(drop=True)

In [40]:
final_stop_words

,و
0,در
1,به
2,از
3,که
4,این
5,را
6,با
7,است
8,برای
9,آن


In [9]:
df.isna().any().any()

False

In [24]:
import pandas as pd
# Assuming df is your DataFrame
df.rename(columns={'body': 'comment'}, inplace=True)

In [43]:
df

,comment,rate
0,جنس خوبی داره من برا تهویه اتاق خواب گرفتم راض...,5
1,خوبه و کار راه اندازه ولی برای پایه هاش چه خوب...,3
2,کاش بزرگتر بود یا چرخش داشت,5
3,دوساله دارمش وخداروشکر مشکلی نداشته وازش راضیم,5
4,بد نیست,3
...,...,...
1986,دوستش داشتم خیلی خوب بود,5
1987,ممنون از دیجی کالا,5
1988,کیفیت خوب,4
1989,خوبه قیمت و کیفیت مناسبی داره جمع و جوره واسه ...,4


In [25]:
df['comment'] = modify_sentences(df['comment'])
df['comment'] = fix_half_space(df['comment'])
df['comment']=remove_stop_words(df['comment'])
df['comment']=remove_consecutive_duplicates(df['comment'])

In [12]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
model_name = "meta-llama/Meta-Llama-3-8B"

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

In [14]:
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=3
)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [16]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [26]:
# Define the mapping for sentiment labels
def assign_label(rate):
    if rate in [0, 1, 2]:
        return 'negative'
    elif rate in [3, 4]:
        return 'neutral'
    elif rate == 5:
        return 'positive'
    else:
        return 'unknown'  # In case of unexpected values

# Assign labels
df['label'] = df['rate'].apply(assign_label)

# Map labels to numerical IDs
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label_id'] = df['label'].map(label_mapping)

print(df[['rate', 'label', 'label_id']].head())

   rate     label  label_id
0     5  positive         2
1     3   neutral         1
2     5  positive         2
3     5  positive         2
4     3   neutral         1


In [27]:
df

,comment,rate,label,label_id
0,جنس خوبی داره من برا تهویه اتاق خواب گرفتم راض...,5,positive,2
1,خوبه و کار راه اندازه ولی پایه هاش چه خوب میشد...,3,neutral,1
2,کاش بزرگتر بود یا چرخش,5,positive,2
3,دوساله دارمش وخداروشکر مشکلی نداشته وازش راضیم,5,positive,2
4,بد نیست,3,neutral,1
...,...,...,...,...
1986,دوستش داشتم خیلی خوب بود,5,positive,2
1987,منون دیجی کالا,5,positive,2
1988,کیفیت خوب,4,neutral,1
1989,خوبه قیمت و کیفیت مناسبی داره جمع و جوره واسه ...,4,neutral,1


In [28]:
df.drop('rate',axis=1 , inplace =True)
df.drop('label',axis=1 , inplace =True)
df

,comment,label_id
0,جنس خوبی داره من برا تهویه اتاق خواب گرفتم راض...,2
1,خوبه و کار راه اندازه ولی پایه هاش چه خوب میشد...,1
2,کاش بزرگتر بود یا چرخش,2
3,دوساله دارمش وخداروشکر مشکلی نداشته وازش راضیم,2
4,بد نیست,1
...,...,...
1986,دوستش داشتم خیلی خوب بود,2
1987,منون دیجی کالا,2
1988,کیفیت خوب,1
1989,خوبه قیمت و کیفیت مناسبی داره جمع و جوره واسه ...,1


In [29]:
train_end_point = int(df.shape[0]*0.6)
val_end_point = int(df.shape[0]*0.8)
df_train = df.iloc[:train_end_point,:]
df_val = df.iloc[train_end_point:val_end_point,:]
df_test = df.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
    'test': dataset_test
})
dataset

(1194, 2) (399, 2) (398, 2)


DatasetDict({
    train: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 1194
    })
    val: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 398
    })
    test: Dataset({
        features: ['comment', 'label_id'],
        num_rows: 399
    })
})

In [31]:
MAX_LEN = 512
col_to_delete = ['comment']

def llama_preprocessing_function(examples):
    return tokenizer(examples['comment'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("label_id", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/1194 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

In [32]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':f1_score(predictions,labels)}

In [40]:
peft_training_args = TrainingArguments(
    output_dir='sentiment_classification',
    #auto_find_batch_size=True,
    per_device_train_batch_size=20,  # Reduce batch size if encountering out of memory error during training
    per_device_eval_batch_size=20,  # Reduce batch size if encountering out of memory error during training
    learning_rate=6e-5,
    num_train_epochs=1,
    logging_steps=1,
    #max_steps=1
    #evaluation_strategy = 'steps',
    #save_strategy = 'steps',
    #load_best_model_at_end = True
)

In [41]:
peft_trainer = Trainer(
    model=model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics
)

In [ ]:
train_result = peft_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,0.425300
2,0.831700
3,0.846000
4,0.520600
5,1.037500
6,0.882000
7,1.083200
8,0.724200
9,1.406600
10,0.637500


Step,Training Loss
1,0.425300
2,0.831700
3,0.846000
4,0.520600
5,1.037500
6,0.882000
7,1.083200
8,0.724200
9,1.406600
10,0.637500


In [ ]:
# Save the model and tokenizer (assuming you've already run the training code)
peft_model_path = "./peft-sentiment-analysis-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

# Import necessary libraries
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from peft import PeftModel

# Load the saved model and tokenizer
peft_model = AutoModelForSequenceClassification.from_pretrained(peft_model_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device)

# Function to get predictions
def get_predictions(model, dataset):
    model.eval()
    predictions, true_labels = [], []
    for example in dataset:
        inputs = {
            'input_ids': example['input_ids'].clone().detach().unsqueeze(0).to(device),
            'attention_mask': example['attention_mask'].clone().detach().unsqueeze(0).to(device)
        }
        with torch.no_grad():
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend([example['label'].item()])
    return predictions, true_labels

# Get predictions on the test set
peft_predictions, test_labels = get_predictions(peft_model, tokenized_datasets['test'])

# Calculate metrics
peft_accuracy = accuracy_score(test_labels, peft_predictions)
peft_precision = precision_score(test_labels, peft_predictions, average='weighted')
peft_recall = recall_score(test_labels, peft_predictions, average='weighted')
peft_f1 = f1_score(test_labels, peft_predictions, average='weighted')
peft_report = classification_report(test_labels, peft_predictions)

# Use Streamlit within Jupyter Notebook
import streamlit as st

def streamlit_app():
    st.title('Sentiment Analysis Model Metrics')

    # Display metrics
    st.header('Performance Metrics')
    st.write(f'**Accuracy:** {peft_accuracy:.4f}')
    st.write(f'**Precision:** {peft_precision:.4f}')
    st.write(f'**Recall:** {peft_recall:.4f}')
    st.write(f'**F1 Score:** {peft_f1:.4f}')

    # Display classification report
    st.header('Classification Report')
    st.text(peft_report)

    # Confusion Matrix
    st.header('Confusion Matrix')
    conf_matrix = confusion_matrix(test_labels, peft_predictions)
    fig, ax = plt.subplots()
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    st.pyplot(fig)

# Run Streamlit app in Jupyter Notebook
from jupyter_streamlit import StreamlitApp
StreamlitApp(streamlit_app).run()

In [ ]:
# استفاده از این کلاس برای خواندن داده ها به شکلی که برای شبکه مناسب باشه

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,  # Adjust as needed
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }

In [ ]:
# تقسیم داده ها به دو ببخش تست و ترین
y= df.rate.values.astype(int)
x= df.comment.values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state= 21)

In [ ]:
# حداکثر بچ سایز 16 می باشد

# Prepare training data
dataset = TextDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Prepare testing data
dataset2 = TextDataset(x_test, y_test)
dataloader_test = DataLoader(dataset2, batch_size=16)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5)

In [ ]:
# آموزش شبکه

num_epochs = 50
for epoch in range(num_epochs):
    model.train()

    total_loss = 0
    correct = 0
    total = 0

    for i, batch in enumerate(dataloader):
        input_ids = batch['input_ids'].to(device)

        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        probabilities = torch.softmax(outputs.logits, dim=1)[0]
        predicted_label = torch.argmax(probabilities).item()

        total += labels.size(0)
        correct += (predicted_label == labels).sum().item()

        print(f'Batch : {i} loss = {loss.item()}')

        accuracy = 100 * correct / total



        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}, accuracy: {accuracy}")

predictions = []
true_labels = []

model.eval()  # Switch to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for batch in dataloader_test:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].cpu().numpy()  # Convert labels to CPU for sklearn

        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs.logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).cpu().numpy()  # Convert predictions to CPU for sklearn

        predictions.extend(predicted_label)
        true_labels.extend(labels)

f1 = f1_score(true_labels, predictions,average= 'weighted')
print(f"F1 score after training: {f1}")
